In [35]:
import numpy as np
from scipy.optimize import dual_annealing
import pulp
import pandas as pd
import os
import random
from wmape import WMAPE
from order_generator import OrderGenerator
from order_merger import OrderMerger
import math
import matplotlib.pyplot as plt

In [36]:
random.seed(42)

In [3]:
# import allocations for lead day 5 and lead day 0 and calculate wmape values

allocation_dir = 'allocations'
merged_dir = 'merged'
lead_day = 5
soft_col = f'lead_{lead_day}'
hard_col = 'lead_0'

allocation_0 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_0.csv')
allocation_5 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_5.csv')

merged_5_0 = pd.read_csv(f'{merged_dir}/merged_lead_day_{lead_day}_0.csv')

_, wmape_site = WMAPE.calculate_wmape_site(merged_5_0, soft_col, hard_col)
_, wmape_global = WMAPE.calculate_wmape_global(merged_5_0, soft_col, hard_col)

print(f'wmape_global: {wmape_global}')
print(f'wmape_site: {wmape_site}')

wmape_global: 0.21075893420886477
wmape_site: 0.42063391842404346


In [27]:
def objective_function(current_allocation, orders_df, target_orders_df, initial_day, target_day):
    # Apply the current allocation to the orders dataframe
    orders_df['site'] = current_allocation

    # Aggregate the orders for the initial and target lead days
    aggregated_initial_df = OrderMerger.aggregate_items(orders_df, initial_day)
    aggregated_target_df = OrderMerger.aggregate_items(target_orders_df, target_day)

    # Merge the aggregated dataframes
    merged_df = OrderMerger.merge_allocation(aggregated_initial_df, aggregated_target_df, initial_day, target_day)

    # Calculate WMAPE
    _, wmape_site = WMAPE.calculate_wmape_site(merged_df, f'lead_{initial_day}', f'lead_{target_day}')
    return wmape_site


In [28]:
def cooling_schedule(initial_temp, alpha, step):
    return initial_temp * (alpha ** step)


def generate_neighbor(current_allocation, orders_df, eligibility_dict):
    new_allocation = current_allocation.copy()
    max_attempts = 100  # Limit the number of attempts to find a valid swap

    for _ in range(max_attempts):
        idx1, idx2 = random.sample(range(len(new_allocation)), 2)

        # Check eligibility for idx1
        order_id1 = orders_df.iloc[idx1]['order_id']
        items1 = orders_df[orders_df['order_id'] == order_id1]['item_id'].tolist()
        site1 = new_allocation[idx2]  # Swap site for idx1

        if all(item in eligibility_dict[site1] for item in items1):
            # Check eligibility for idx2
            order_id2 = orders_df.iloc[idx2]['order_id']
            items2 = orders_df[orders_df['order_id'] == order_id2]['item_id'].tolist()
            site2 = new_allocation[idx1]  # Swap site for idx2

            if all(item in eligibility_dict[site2] for item in items2):
                # Perform the swap if both are eligible
                new_allocation[idx1], new_allocation[idx2] = new_allocation[idx2], new_allocation[idx1]
                break

    return new_allocation



def acceptance_probability(current_energy, new_energy, temperature):
    if new_energy < current_energy:
        return 1.0
    else:
        return math.exp((current_energy - new_energy) / temperature)


def simulated_annealing(orders_df, target_orders_df, eligibility_dict, initial_day, target_day, initial_temp, final_temp, alpha, max_iter):
    current_allocation = orders_df['site'].values
    best_allocation = current_allocation.copy()
    current_energy = objective_function(current_allocation, orders_df, target_orders_df, initial_day, target_day)
    best_energy = current_energy

    temp = initial_temp
    step = 0

    while temp > final_temp and step < max_iter:
        new_allocation = generate_neighbor(current_allocation, orders_df, eligibility_dict)
        new_energy = objective_function(new_allocation, orders_df, target_orders_df, initial_day, target_day)

        if acceptance_probability(current_energy, new_energy, temp) > random.random():
            current_allocation = new_allocation
            current_energy = new_energy

            if current_energy < best_energy:
                best_allocation = current_allocation
                best_energy = current_energy

        temp = cooling_schedule(initial_temp, alpha, step)
        step += 1

    return best_allocation, best_energy

In [31]:
# Define the parameters for simulated annealing
initial_temp = 5000
final_temp = 1
alpha = 0.95
max_iter = 1000000000

eligibility_dict = OrderGenerator.load_eligibility_dict('eligibility_dict.json')

# Define the lead days for initial and target allocations
initial_day = 0
target_day = 5

# Run the simulated annealing optimization
best_allocation, best_energy = simulated_annealing(allocation_0.copy(), allocation_5.copy(), eligibility_dict, initial_day, target_day, initial_temp, final_temp, alpha, max_iter)

new_allocation = allocation_0.copy()
# Apply the best allocation to the orders dataframe
new_allocation['site'] = best_allocation

print("Best WMAPE:", best_energy)
print("Best Allocation:", best_allocation)

Best WMAPE: 0.41937113271877763
Best Allocation: ['F20' 'F20' 'F20' ... 'F20' 'F20' 'F20']


In [37]:
allocation_dir = 'allocations'

allocation_0 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_0.csv')
allocation_5 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_5.csv')
allocation_10 = pd.read_csv(f'{allocation_dir}/allocation_lead_day_10.csv')

In [38]:
# Define the parameters for simulated annealing
initial_temp = 5000
final_temp = 1
alpha = 0.95
max_iter = 1000000000

eligibility_dict = OrderGenerator.load_eligibility_dict('eligibility_dict.json')

initial_day_10 = 10
target_day_5 = 5
best_allocation_5, best_energy_5 = simulated_annealing(allocation_10.copy(), allocation_5.copy(), eligibility_dict, initial_day_10, target_day_5, initial_temp, final_temp, alpha, max_iter)

# Update the allocation for lead day 5 with the best found allocation
allocation_5['site'] = best_allocation_5


# Perform simulated annealing between lead day 5 and lead day 0
initial_day_5 = 5
target_day_0 = 0
best_allocation_0, best_energy_0 = simulated_annealing(allocation_5.copy(), allocation_0.copy(), eligibility_dict, initial_day_5, target_day_0, initial_temp, final_temp, alpha, max_iter)

# Update the allocation for lead day 0 with the best found allocation
allocation_0['site'] = best_allocation_0


KeyError: 'lead_0'

In [ ]:
# Aggregate and merge the updated allocation dataframes with the original lead day 0 allocation
aggregated_10 = OrderMerger.aggregate_items(allocation_10, 10)
aggregated_5 = OrderMerger.aggregate_items(allocation_5, 5)
aggregated_0 = OrderMerger.aggregate_items(allocation_0, 0)

merged_10_0 = OrderMerger.merge_allocation(aggregated_10, aggregated_0, 10, 0)
merged_5_0 = OrderMerger.merge_allocation(aggregated_5, aggregated_0, 5, 0)

# Calculate WMAPE site and global for each merged dataframe
site_10_0_df, wmape_10_0_site = WMAPE.calculate_wmape_site(merged_10_0, 'lead_10', 'lead_0')
global_10_0_df, wmape_10_0_global = WMAPE.calculate_wmape_global(merged_10_0, 'lead_10', 'lead_0')

site_5_0_df, wmape_5_0_site = WMAPE.calculate_wmape_site(merged_5_0, 'lead_5', 'lead_0')
global_5_0_df, wmape_5_0_global = WMAPE.calculate_wmape_global(merged_5_0, 'lead_5', 'lead_0')


In [ ]:
# Data for plotting
lead_days = [10, 5, 0]
wmape_site_values = [wmape_10_0_site, wmape_5_0_site, 0]
wmape_global_values = [wmape_10_0_global, wmape_5_0_global, 0]

# Plot the data
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(lead_days, wmape_site_values, linestyle='-', color='b', label='WMAPE Site')
ax.plot(lead_days, wmape_global_values, linestyle='-', color='r', label='WMAPE Global')

# Add labels and title
ax.set_xlabel('Lead Day')
ax.set_ylabel('Error')
ax.set_title('WMAPE Values for Different Lead Days')
ax.set_xticks([10, 5, 0])
ax.legend()

# Invert x-axis to have 0 at the end
ax.invert_xaxis()

plt.show()
